In [ ]:
-- Active: 1660012487192@@localhost@5432@Covid_Project
SELECT * FROM covid_deaths
limit 10
-- SELECT * FROM covid_vaccination
-- limit 10

In [ ]:
-- Active: 1660012487192@@localhost@5432@Covid_Project
SELECT location, date, total_cases, new_cases, total_deaths, population
FROM covid_deaths
ORDER BY 1,2
limit 10

In [ ]:
-- Active: 1660012487192@@localhost@5432@Covid_Project
-- melihat jumlah kasus vs jumlah kematian
-- melihat persentase jumlah kematian akibat kasus covid
SELECT location, date, total_cases,  total_deaths, (total_deaths / total_cases)*100 AS DeathPercentage
FROM covid_deaths
ORDER BY 1,2
limit 10


In [ ]:
-- Active: 1660012487192@@localhost@5432@Covid_Project
-- melihat jumlah kasus vs populasi
-- menampilkan persentase populasi yang terkena covid

SELECT location, date,  total_cases, population, CAST((total_cases/CAST(population AS DECIMAL (13,2))*100) AS DECIMAL) AS PopulasiTerinfeksipercentage
FROM covid_deaths
WHERE location = 'World'
 ORDER BY 1,2


In [ ]:
-- Active: 1660012487192@@localhost@5432@Covid_Project
-- melihat negara dengan total kasus covid terbanyak

SELECT location, population,  MAX(total_cases) AS HighestInfectionCountries, MAX(CAST((total_cases/CAST(population AS DECIMAL (13,2))*100) AS DECIMAL)) AS persentasepopulasiterinfeksi
FROM covid_deaths
--biar gk ada nullnya
WHERE population IS NOT NULL AND total_cases IS NOT NULL

GROUP BY location, population

ORDER BY persentasepopulasiterinfeksi DESC


In [ ]:
-- Active: 1660012487192@@localhost@5432@Covid_Project
-- melihat negara dengan kematian terbanyak
SELECT location, MAX(total_deaths) AS totalDeathCount
FROM covid_deaths
WHERE  total_deaths IS NOT NULL
AND location <> continent
GROUP BY location
ORDER BY totalDeathCount DESC
limit 10

In [ ]:
-- Active: 1660012487192@@localhost@5432@Covid_Project
-- melihat benua dengan kematian terbanyak
SELECT continent, location, MAX(total_deaths) AS totalDeathCount
FROM covid_deaths
WHERE  continent IS NOT NULL
GROUP BY continent, location
ORDER BY totalDeathCount DESC
--limit 10

In [ ]:
-- Active: 1660012487192@@localhost@5432@Covid_Project
SELECT date, total_cases, total_deaths, (total_deaths/total_cases)*100 as kematianperkasus
FROM covid_deaths
WHERE  total_cases IS NOT NULL
AND total_deaths IS NOT NULL



In [ ]:
-- Active: 1660012487192@@localhost@5432@Covid_Project
--manampilkan persentasi total kasus baru VS total kasus kematian baru
--Nomor Global
SELECT date, SUM(COALESCE(new_cases,0)) AS Total_kasus_Covid, 
SUM(COALESCE(new_deaths,0)) AS Total_Kematian_Baru, 
SUM(new_deaths)/NULLIF(SUM(new_cases),0) AS Persentase_Kematian
FROM covid_deaths
GROUP BY date
ORDER BY 1,2

In [ ]:
-- Active: 1660012487192@@localhost@5432@Covid_Project
--menampilkan tabel covid_vaccinatio
SELECT * FROM covid_vaccination

In [ ]:
-- Active: 1660012487192@@localhost@5432@Covid_Project
--mengabungkan tabel covid_date dan covid_vaccination
SELECT C_deaths.continent, C_deaths.location, C_deaths.date, C_deaths.population, C_vaccin.new_vaccinations,
SUM(C_vaccin.new_vaccinations) OVER (PARTITION by C_deaths.location ORDER BY C_deaths.date) AS Jumlah_Orang_Vacin_Perwaktu
FROM covid_deaths AS C_deaths
FULL OUTER JOIN covid_vaccination AS C_vaccin
ON C_deaths.location = C_vaccin.location
AND C_deaths.date = C_vaccin.date
ORDER BY 2,3

In [ ]:
-- Active: 1660012487192@@localhost@5432@Covid_Project
--membuat CTE
WITH PopvsVac (continent, location, date, population, new_vaccinations, Jumlah_Orang_Vacin_Perwaktu)
AS
(
SELECT C_deaths.continent, C_deaths.location, C_deaths.date, C_deaths.population, C_vaccin.new_vaccinations,
SUM(C_vaccin.new_vaccinations) OVER (PARTITION by C_deaths.location ORDER BY C_deaths.date) AS Jumlah_Orang_Vacin_Perwaktu
FROM covid_deaths AS C_deaths
FULL OUTER JOIN covid_vaccination AS C_vaccin
ON C_deaths.location = C_vaccin.location
AND C_deaths.date = C_vaccin.date
)

SELECT *, (Jumlah_Orang_Vacin_Perwaktu/population)*100 AS Jumlah_vaksin_Per_Popolasi
FROM PopvsVac

In [ ]:
-- Active: 1660012487192@@localhost@5432@Covid_Project
--membuat TABLE
-- CREATE TABLE IF NOT EXISTS public.Presentase_Populasi_Tervaksin
-- (
--     continent nvarchar(225),
--     location nvarchar(225),
--     date datetime,
--     population numeric,
--     new_vaccinations numeric,
--     Jumlah_vaksin_Per_Popolasi numeric
-- )

-- INSERT INTO Presentase_Populasi_Tervaksin
-- SELECT C_deaths.continent, C_deaths.location, C_deaths.date, C_deaths.population, C_vaccin.new_vaccinations,
-- SUM(C_vaccin.new_vaccinations) OVER (PARTITION by C_deaths.location ORDER BY C_deaths.date) AS Jumlah_Orang_Vacin_Perwaktu
-- FROM covid_deaths AS C_deaths
-- FULL OUTER JOIN covid_vaccination AS C_vaccin
-- ON C_deaths.location = C_vaccin.location
-- AND C_deaths.date = C_vaccin.date

-- SELECT *, (Jumlah_Orang_Vacin_Perwaktu/population)*100 AS Jumlah_vaksin_Per_Popolasi
-- FROM Pesentase_Populasi_Tervaksin


In [ ]:
-- Active: 1660012487192@@localhost@5432@Covid_Project
--membuat VIEW
CREATE VIEW Jumlah_vaksin_Per_Popolasi AS
SELECT C_deaths.continent, C_deaths.location, C_deaths.date, C_deaths.population, C_vaccin.new_vaccinations,
SUM(C_vaccin.new_vaccinations) OVER (PARTITION by C_deaths.location ORDER BY C_deaths.date) AS Jumlah_Orang_Vacin_Perwaktu
FROM covid_deaths AS C_deaths
FULL OUTER JOIN covid_vaccination AS C_vaccin
ON C_deaths.location = C_vaccin.location
AND C_deaths.date = C_vaccin.date

In [ ]:
-- Active: 1660012487192@@localhost@5432@Covid_Project
SELECT * FROM Jumlah_vaksin_Per_Popolasi